<a href="https://colab.research.google.com/github/addinar/permafrost-modeling-convlstm/blob/main/data/notebooks/rcp_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Download Packages and Import Libraries**

In [12]:
!pip install xarray netCDF4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 70.9 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import os

In [13]:
import xarray as xr

In [5]:
from google.colab import drive, userdata
drive.mount('/content/drive')

Mounted at /content/drive


In [192]:
rcp_path = userdata.get('rcp_output_path')
rcp26_path = os.path.join(rcp_path, 'RCP26_df.csv')
rcp45_path = os.path.join(rcp_path, 'RCP45_df.csv')
rcp60_path = os.path.join(rcp_path, 'RCP60_df.csv')
rcp85_path = os.path.join(rcp_path, 'RCP85_df.csv')

In [193]:
rcp26_df = pd.read_csv(rcp26_path)
rcp45_df = pd.read_csv(rcp45_path)
rcp60_df = pd.read_csv(rcp60_path)
rcp85_df = pd.read_csv(rcp85_path)

In [194]:
dfs = [rcp26_df, rcp45_df, rcp60_df, rcp85_df]

In [171]:
# first inspect column names for all
for df in dfs:
  print(df.columns)

Index(['Unnamed: 0', 'time', 'snow_depth', 'band', 'skin_temperature',
       'temperature_2m', 'snowfall_sum',
       'surface_thermal_radiation_downwards_sum',
       'avg_volumetric_water_content', 'average_lake_temperature',
       'total_precipitation_sum', 'surface_latent_heat_flux_sum',
       'surface_sensible_heat_flux_sum', 'surface_snow_amount',
       'surface_solar_radiation_downwards_sum'],
      dtype='object')
Index(['Unnamed: 0', 'time', 'surface_solar_radiation_downwards_sum', 'band',
       'surface_thermal_radiation_downwards_sum', 'total_precipitation_sum',
       'surface_sensible_heat_flux_sum', 'snowfall_sum', 'surface_snow_amount',
       'skin_temperature', 'temperature_2m', 'surface_latent_heat_flux_sum',
       'snow_depth', 'average_lake_temperature',
       'avg_volumetric_water_content'],
      dtype='object')
Index(['Unnamed: 0', 'time', 'surface_latent_heat_flux_sum', 'band',
       'surface_solar_radiation_downwards_sum', 'avg_volumetric_water_content'

# **Derive Additional Features**

## **Snow Cover & Snow Density**

In [172]:
rcp26_df.columns

Index(['Unnamed: 0', 'time', 'snow_depth', 'band', 'skin_temperature',
       'temperature_2m', 'snowfall_sum',
       'surface_thermal_radiation_downwards_sum',
       'avg_volumetric_water_content', 'average_lake_temperature',
       'total_precipitation_sum', 'surface_latent_heat_flux_sum',
       'surface_sensible_heat_flux_sum', 'surface_snow_amount',
       'surface_solar_radiation_downwards_sum'],
      dtype='object')

snow_cover = min(1, $\frac{1000 \times SD}{15}$)

where SD = snow density

snow density = $\frac{snw}{snd}$

source for this equation can be found [here](https://confluence.ecmwf.int/display/CKB/ERA-Interim%3A+documentation).



In [174]:
dfs = [rcp26_df, rcp45_df, rcp60_df, rcp85_df]

In [196]:
MIN_SNOW_DEPTH = 0.01  # 1 cm
MIN_SNOW_AMOUNT = 0.1  # 0.1 kg/m²

for df in dfs:
    df['surface_snow_amount'] = df['surface_snow_amount'].apply(lambda x: x if x >= MIN_SNOW_AMOUNT else float('nan'))
    df['snow_depth'] = df['snow_depth'].apply(lambda x: x if x >= MIN_SNOW_DEPTH else float('nan'))

    df['snow_density'] = df['surface_snow_amount'] / df['snow_depth']

    df['snow_cover'] = df['snow_density'].apply(lambda x: min(1, (1000 * x) / 15) if not pd.isna(x) else float('nan'))

## **Snow Albedo**

Snow albedo formula:
$$
Albedo \approx \frac{rsus}{rsds}
$$

In [202]:
# first we need to get rsus (surface shortwave upwelling radiation) and process into a dataframe
rcp_path = userdata.get('RCP_folder')
rcp26_rsus = os.path.join(rcp_path, 'RCP26_rsus')
rcp45_rsus = os.path.join(rcp_path, 'RCP45_rsus')
rcp60_rsus = os.path.join(rcp_path, 'RCP60_rsus')
rcp85_rsus = os.path.join(rcp_path, 'RCP85_rsus')

In [203]:
latitudes = {
    '0': [59.66292135, 61.68539326], # band 1: [min_lat, max_lat]
    '1': [61.68539326, 63.70786517], # band 2
    '2': [63.70786517, 65.73033708], # band 3
    '3': [65.73033708, 67.75280899], # band 4
    '4': [67.75280899, 69.7752809], # band 5
    '5': [69.7752809, 71.79775281] # band 6
}

In [204]:
# inspect one
os.listdir(rcp26_rsus)

['rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_206601-207012.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_206101-206512.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_209101-209512.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_208101-208512.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_208601-209012.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_207101-207512.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_209601-210012.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_205601-206012.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_207601-208012.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_203101-203512.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_205101-205512.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_203601-204012.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_204601-205012.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_204101-204512.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_202101-202512.nc',
 'rsus_Amon_GFDL-ESM2M_rcp26_r1i1p1_202601-203012.nc']

In [205]:
def create_df(file_name, rcp_folder):
  full_path = os.path.join(rcp_folder, file_name)
  data = xr.open_dataset(full_path)
  data = data.assign_coords(
    lon=(((data.lon + 180) % 360) - 180)
  )
  file_dfs = []
  for i in range(6):
    lats = latitudes[str(i)]
    min_lat = lats[0]
    max_lat = lats[1]
    try:
      region = data.sel(
        lat=slice(min_lat, max_lat),
        lon=slice(-168.75, -143.75)
      )
    except:
      region = data.sel(
        rlat=slice(min_lat, max_lat),
        rlon=slice(-168.75, -143.75)
      )
    df = region['rsus'].to_dataframe().reset_index()
    df = pd.DataFrame(df.groupby('time')['rsus'].max()).reset_index()
    df['band'] = f'band_{i+1}'
    file_dfs.append(df)
  file_df = pd.concat(file_dfs, axis=0).reset_index()
  return file_df

In [206]:
rcp26_rsus_data = []
rcp45_rsus_data = []
rcp60_rsus_data = []
rcp85_rsus_data = []

In [207]:
for file in os.listdir(rcp26_rsus):
  file_df = create_df(file, rcp26_rsus)
  rcp26_rsus_data.append(file_df)

rcp_26_rsus_df = pd.concat(rcp26_rsus_data, axis=0).reset_index(drop=True)

<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  data = xr.open_dataset(full_path)
<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  data = xr.open_dataset(full_path)
<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  data = xr.open_dataset(full_path)
<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to Tru

In [208]:
rcp_26_rsus_df.shape

(5760, 4)

In [209]:
for file in os.listdir(rcp45_rsus):
  file_df = create_df(file, rcp45_rsus)
  rcp45_rsus_data.append(file_df)

rcp_45_rsus_df = pd.concat(rcp45_rsus_data, axis=0).reset_index(drop=True)

<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  data = xr.open_dataset(full_path)
<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  data = xr.open_dataset(full_path)
<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  data = xr.open_dataset(full_path)
<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to Tru

In [210]:
rcp_45_rsus_df.shape

(5760, 4)

In [211]:
for file in os.listdir(rcp60_rsus):
  file_df = create_df(file, rcp60_rsus)
  rcp60_rsus_data.append(file_df)

rcp_60_rsus_df = pd.concat(rcp60_rsus_data, axis=0).reset_index(drop=True)

<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  data = xr.open_dataset(full_path)
<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  data = xr.open_dataset(full_path)
<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  data = xr.open_dataset(full_path)
<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to Tru

In [212]:
rcp_60_rsus_df.shape

(5760, 4)

In [213]:
for file in os.listdir(rcp85_rsus):
  file_df = create_df(file, rcp85_rsus)
  rcp85_rsus_data.append(file_df)

rcp_85_rsus_df = pd.concat(rcp85_rsus_data, axis=0).reset_index(drop=True)

<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  data = xr.open_dataset(full_path)
<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  data = xr.open_dataset(full_path)
<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  data = xr.open_dataset(full_path)
<ipython-input-205-f3db4d95da15>:3: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to Tru

In [214]:
rcp_85_rsus_df.shape

(5760, 4)

In [215]:
rcp26_df['time'] = pd.to_datetime(rcp26_df['time'].astype(str)).dt.date
rcp45_df['time'] = pd.to_datetime(rcp45_df['time'].astype(str)).dt.date
rcp60_df['time'] = pd.to_datetime(rcp60_df['time'].astype(str)).dt.date
rcp85_df['time'] = pd.to_datetime(rcp85_df['time'].astype(str)).dt.date

rcp_26_rsus_df['time'] = pd.to_datetime(rcp_26_rsus_df['time'].astype(str)).dt.date
rcp_45_rsus_df['time'] = pd.to_datetime(rcp_45_rsus_df['time'].astype(str)).dt.date
rcp_60_rsus_df['time'] = pd.to_datetime(rcp_60_rsus_df['time'].astype(str)).dt.date
rcp_85_rsus_df['time'] = pd.to_datetime(rcp_85_rsus_df['time'].astype(str)).dt.date

In [216]:
rcp26_df = rcp26_df.merge(rcp_26_rsus_df, on=['time', 'band'], how='outer')
rcp45_df = rcp45_df.merge(rcp_45_rsus_df, on=['time', 'band'], how='outer')
rcp60_df = rcp60_df.merge(rcp_60_rsus_df, on=['time', 'band'], how='outer')
rcp85_df = rcp85_df.merge(rcp_85_rsus_df, on=['time', 'band'], how='outer')

In [217]:
print(rcp26_df.isna().sum(), rcp45_df.isna().sum(), rcp60_df.isna().sum(), rcp85_df.isna().sum())

Unnamed: 0                                    0
time                                          0
snow_depth                                 2727
band                                          0
skin_temperature                              0
temperature_2m                                0
snowfall_sum                                  0
surface_thermal_radiation_downwards_sum       0
avg_volumetric_water_content                  0
average_lake_temperature                      0
total_precipitation_sum                       0
surface_latent_heat_flux_sum                  0
surface_sensible_heat_flux_sum                0
surface_snow_amount                        1134
surface_solar_radiation_downwards_sum         0
snow_density                               2737
snow_cover                                 2737
index                                         0
rsus                                          0
dtype: int64 Unnamed: 0                                    0
time                       

In [218]:
# rename dfs
dfs = [rcp26_df, rcp45_df, rcp60_df, rcp85_df]

In [225]:
for df in dfs:
    snow_covered_df = df.loc[df['snow_cover'] == 1].copy()
    snow_covered_df.loc[:, 'snow_albedo'] = snow_covered_df['rsus'] / snow_covered_df['surface_solar_radiation_downwards_sum']
    snow_covered_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    snow_covered_df.fillna(0, inplace=True)
    df.loc[df['snow_cover'] == 1, 'snow_albedo'] = snow_covered_df['snow_albedo']

In [226]:
dfs = [rcp26_df, rcp45_df, rcp60_df, rcp85_df]

for df in dfs:
  df = df.fillna(0)

In [227]:
rcp26_df.shape

(5760, 20)

# **Impute Missing Data**

In [229]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [230]:
imputer = IterativeImputer(max_iter=10, random_state=0)

In [231]:
dfs = [rcp26_df, rcp45_df, rcp60_df, rcp85_df]

In [232]:
for df in dfs:
  missing_cols = df.columns[df.isna().any()]
  df[missing_cols] = imputer.fit_transform(df[missing_cols])
  print(df.isna().sum())

Unnamed: 0                                 0
time                                       0
snow_depth                                 0
band                                       0
skin_temperature                           0
temperature_2m                             0
snowfall_sum                               0
surface_thermal_radiation_downwards_sum    0
avg_volumetric_water_content               0
average_lake_temperature                   0
total_precipitation_sum                    0
surface_latent_heat_flux_sum               0
surface_sensible_heat_flux_sum             0
surface_snow_amount                        0
surface_solar_radiation_downwards_sum      0
snow_density                               0
snow_cover                                 0
index                                      0
rsus                                       0
snow_albedo                                0
dtype: int64
Unnamed: 0                                 0
time                                      

# **Restructure Dataframes**

In [233]:
dfs = [rcp26_df, rcp45_df, rcp60_df, rcp85_df]

In [234]:
new_order = ['date', 'band', 'snow_albedo', 'snow_cover', 'snow_density',
             'snow_depth', 'snow_depth_water_equivalent',
             'snowfall_sum', 'surface_latent_heat_flux_sum',
             'surface_sensible_heat_flux_sum', 'surface_solar_radiation_downwards_sum',
             'surface_thermal_radiation_downwards_sum', 'skin_temperature',
             'temperature_2m', 'total_precipitation_sum', 'avg_volumetric_water_content']

In [235]:
bands = {
    'band_1' : np.mean([61.2755545, 59.2632802]),
    'band_2' : np.mean([63.2878288, 61.2755545]),
    'band_3' : np.mean([65.3001031, 63.2878288]),
    'band_4' : np.mean([65.3001031, 67.3123774]),
    'band_5' : np.mean([67.3123774, 69.3246517]),
    'band_6' : np.mean([69.3246517, 71.336926])
}

In [237]:
for i, df in enumerate(dfs):
    dfs[i] = df.copy()
    dfs[i].rename(columns={'time': 'date', 'surface_snow_amount': 'snow_depth_water_equivalent'}, inplace=True)
    dfs[i] = dfs[i][new_order]
    for key, value in bands.items():
        dfs[i].loc[dfs[i]['band'] == key, 'average_latitude'] = value

# **Z-Score Normalization**

In [239]:
dfs = [rcp26_df, rcp45_df, rcp60_df, rcp85_df]

In [240]:
scaler = StandardScaler()

In [241]:
normalized_dfs = []

In [242]:
for df in dfs:
  numeric_cols = df.select_dtypes(include=['number']).columns
  numeric_df = df[numeric_cols]
  rest = df[['date', 'band']]
  scaled = scaler.fit_transform(numeric_df)
  scaled_df = pd.DataFrame(scaled, columns=numeric_cols)
  df = pd.concat([rest, scaled_df], axis=1)
  normalized_dfs.append(df)

In [243]:
rcp26_df = normalized_dfs[0]
rcp45_df = normalized_dfs[1]
rcp60_df = normalized_dfs[2]
rcp85_df = normalized_dfs[3]

In [244]:
clean_dfs_path = os.path.join(rcp_path, 'clean_dfs')

rcp26_df.to_csv(os.path.join(clean_dfs_path, 'rcp26_df_processed.csv'))
rcp45_df.to_csv(os.path.join(clean_dfs_path, 'rcp45_df_processed.csv'))
rcp60_df.to_csv(os.path.join(clean_dfs_path, 'rcp60_df_processed.csv'))
rcp85_df.to_csv(os.path.join(clean_dfs_path, 'rcp85_df_processed.csv'))